In [3]:
import numpy as np

In [4]:
from phevaluator.evaluator import evaluate_cards

p1 = evaluate_cards("2c", "3c", "4c", "5c", "As", "8s", "9s")
p2 = evaluate_cards("2c", "3c", "4c", "5c", "7s", "8s", "9s")

# Player 2 has a stronger hand

In [5]:
from deck.card import Card, Rank, Suit
from deck.deck import Deck

In [6]:
def sort_hand(hand: np.ndarray) -> np.ndarray:
    return np.array(sorted(hand, reverse=True, key=lambda x: (x.get_rank(), x.get_tag())))

In [7]:
def is_a_pair(hand: np.ndarray) -> bool:
    return hand[0].get_rank() == hand[1].get_rank()

def compare_starting_hands(hand_one: np.ndarray, hand_two: np.ndarray) -> int:
    """
    Return 1 if hand one is stronger than hand two, -1 if hand two stronger, 0 if equal
    """
    is_hand_one_a_pair = is_a_pair(hand_one)
    is_hand_two_a_pair = is_a_pair(hand_two)
    if is_hand_one_a_pair and not is_hand_two_a_pair:
        return 1
    elif not is_hand_one_a_pair and is_hand_two_a_pair:
        return -1
    elif is_hand_one_a_pair and is_hand_two_a_pair:
        rank_one = hand_one[0].get_rank()
        rank_two = hand_two[0].get_rank()
        if rank_one == rank_two:
            return 0
        if rank_one > rank_two:
            return 1
        else:
            return -1
    else:
        hand_one = sort_hand(hand_one)
        hand_two = sort_hand(hand_two)
        if hand_one[0].get_rank() > hand_two[0].get_rank():
            return 1
        elif hand_one[0].get_rank() < hand_two[0].get_rank():
            return -1
        else:
            if hand_one[1].get_rank() > hand_two[1].get_rank():
                return 1
            elif hand_one[1].get_rank() < hand_two[1].get_rank():
                return -1
            else:
                return 0

In [8]:
def compute_initial_effective_hand_strengths(iterations: int, suited=False):
    if suited:
        shift = 1
        name = "s"
        second_suit = "c"
    else:
        shift = 0
        name = "o"
        second_suit = "s"
    effective_hand_strengths = {}
    for i, rank_one in enumerate(list(Rank)[::-1]):
        for rank_two in list(Rank)[::-1][i+shift:]:
            deck = Deck()
            cards = np.concatenate((deck.draw_card_by_tag(str(rank_one.value) + "c"), deck.draw_card_by_tag(str(rank_two.value) + second_suit)))
            wins = 0
            positive_potential = 0
            negative_potential = 0
            for k in range(iterations):
                deck.shuffle()
                opponent_cards = deck.draw_cards(2)
                initial_advantage = compare_starting_hands(sort_hand(cards), sort_hand(opponent_cards))
                flop = deck.draw_cards(3)
                turn = deck.draw_cards(1)
                river = deck.draw_cards(1)
                my_strength = evaluate_cards(*[card.get_tag() for card in np.concatenate((cards, flop, turn, river))])
                opponent_strength = evaluate_cards(*[card.get_tag() for card in np.concatenate((opponent_cards, flop, turn, river))])
                if my_strength < opponent_strength:
                    wins += 1
                    if initial_advantage == -1:
                        positive_potential += 1 #won the hand with initial disadvantage
                    elif initial_advantage == 0:
                        positive_potential += 0.5
                elif my_strength == opponent_strength: # draw
                    wins += 0.5
                    if initial_advantage == -1:
                        positive_potential += 0.5
                    elif initial_advantage == 1:
                        negative_potential += 0.5
                else: 
                    if initial_advantage == 1: #lost the hand with initial advantage
                        negative_potential += 1
                    elif initial_advantage == 0:
                        negative_potential += 0.5
                deck.return_cards(opponent_cards)
                deck.return_cards(flop)
                deck.return_cards(turn)
                deck.return_cards(river)
            effective_hand_strengths[str(rank_one.value) + str(rank_two.value) + name] = (wins * (iterations - negative_potential) + (iterations - wins) * positive_potential) / iterations ** 2
    return effective_hand_strengths

In [9]:
iterations = 100_000
suited_effective_hand_strength = compute_initial_effective_hand_strengths(iterations, True)
offsuit_effective_hand_strength = compute_initial_effective_hand_strengths(iterations, False)
effective_hand_strength = {
    **suited_effective_hand_strength,
    **offsuit_effective_hand_strength
}
for k,v in sorted(effective_hand_strength.items(), key=lambda item: item[1], reverse=True):
    print(k, v)

AAo 0.725965082875
KKo 0.678422618225
QQo 0.6410408953
54s 0.6283910673
65s 0.624642675325
76s 0.6174152124
53s 0.6157683248
87s 0.61479867195
JJo 0.6102447037
43s 0.6097577808
64s 0.603667434075
75s 0.603393564275
98s 0.59991635275
86s 0.59776195365
52s 0.592415743125
42s 0.59144847905
63s 0.590535007625
74s 0.5894668814
32s 0.588535743425
T9s 0.5882059198
97s 0.584947747
54o 0.58491496425
76o 0.584138493325
65o 0.5826856142
85s 0.58100191
87o 0.5806196061
62s 0.576874136925
TTo 0.574220404125
T8s 0.5741334556
98o 0.57082955335
JTs 0.5707841645
96s 0.570280707125
53o 0.56961441315
43o 0.5691839432
73s 0.5681961762
75o 0.566462057325
84s 0.5656378123
64o 0.5647967464
86o 0.5617588438
T7s 0.5610289053
95s 0.559913271475
T9o 0.55912842
97o 0.5586515648
72s 0.5553314848
J9s 0.5539084433
63o 0.552372879
74o 0.5509358881
82s 0.550111621075
83s 0.5495907521
52o 0.54792350875
85o 0.5469843004
JTo 0.545487109375
42o 0.5454635278
T8o 0.54506342755
94s 0.5449397747
T6s 0.542150518425
J8s 0.54188

In [11]:
for k,v in sorted(effective_hand_strength.items(), key=lambda item: item[1], reverse=True):
    print(k,round(v, 2))

AAo 0.73
KKo 0.68
QQo 0.64
54s 0.63
65s 0.62
76s 0.62
53s 0.62
87s 0.61
JJo 0.61
43s 0.61
64s 0.6
75s 0.6
98s 0.6
86s 0.6
52s 0.59
42s 0.59
63s 0.59
74s 0.59
32s 0.59
T9s 0.59
97s 0.58
54o 0.58
76o 0.58
65o 0.58
85s 0.58
87o 0.58
62s 0.58
TTo 0.57
T8s 0.57
98o 0.57
JTs 0.57
96s 0.57
53o 0.57
43o 0.57
73s 0.57
75o 0.57
84s 0.57
64o 0.56
86o 0.56
T7s 0.56
95s 0.56
T9o 0.56
97o 0.56
72s 0.56
J9s 0.55
63o 0.55
74o 0.55
82s 0.55
83s 0.55
52o 0.55
85o 0.55
JTo 0.55
42o 0.55
T8o 0.55
94s 0.54
T6s 0.54
J8s 0.54
32o 0.54
QJs 0.54
QTs 0.54
93s 0.54
96o 0.54
92s 0.54
99o 0.53
62o 0.53
T5s 0.53
84o 0.53
T7o 0.53
T4s 0.53
73o 0.53
J9o 0.53
J7s 0.53
Q9s 0.52
T3s 0.52
95o 0.52
QJo 0.52
T2s 0.52
QTo 0.52
KQs 0.51
J6s 0.51
J8o 0.51
T6o 0.51
72o 0.51
J5s 0.51
KJs 0.51
83o 0.51
J4s 0.51
Q8s 0.51
82o 0.5
94o 0.5
93o 0.5
KTs 0.5
J3s 0.5
88o 0.5
T5o 0.5
J2s 0.5
J7o 0.5
Q9o 0.5
92o 0.5
T4o 0.49
Q7s 0.49
Q6s 0.49
T3o 0.49
KQo 0.49
K9s 0.49
Q5s 0.49
T2o 0.48
KJo 0.48
Q8o 0.48
Q4s 0.48
AKs 0.48
J6o 0.48
J5o 0.4

In [20]:
import itertools


In [21]:
def compare_hands_after_flop(hand_one: np.ndarray, hand_two: np.ndarray) -> int:
    hand_one_strength = evaluate_cards(*[card.get_tag() for card in hand_one])
    hand_two_strength = evaluate_cards(*[card.get_tag() for card in hand_two])
    if hand_one_strength < hand_two_strength:
        return 1
    elif hand_one_strength == hand_one_strength:
        return 0
    else:
        return -1

In [60]:
def compute_mono_flop_effective_hand_strengths_for_suited_cards(iterations: int, made_flush=True):
    shift = 1
    name = "s"
    if made_flush:
        flush = "f"
        second_suit = "c"
    else:
        flush = "n"
        second_suit = "s"
    effective_hand_strengths = {}
    for i, rank_one in enumerate(list(Rank)[::-1]):
        for rank_two in list(Rank)[::-1][i + shift :]:
            deck = Deck()
            cards = np.concatenate(
                (
                    deck.draw_card_by_tag(str(rank_one.value) + "c"),
                    deck.draw_card_by_tag(str(rank_two.value) + "c"),
                )
            )
            rank_list = list(Rank)
            if made_flush:
                rank_list.remove(rank_one)
                rank_list.remove(rank_two)
            possible_combinations = itertools.combinations(rank_list[::-1], 3)
            for combination in possible_combinations:
                wins = 0
                positive_potential = 0
                negative_potential = 0
                flop = np.concatenate(
                    (
                        deck.draw_card_by_tag(str(combination[0].value) + second_suit),
                        deck.draw_card_by_tag(str(combination[1].value) + second_suit),
                        deck.draw_card_by_tag(str(combination[2].value) + second_suit),
                    )
                )
                for _ in range(iterations):
                    deck.shuffle()
                    opponent_cards = deck.draw_cards(2)
                    initial_advantage = compare_hands_after_flop(
                        np.concatenate((cards, flop)),
                        np.concatenate((opponent_cards, flop)),
                    )
                    turn = deck.draw_cards(1)
                    river = deck.draw_cards(1)
                    final_result = compare_hands_after_flop(
                        np.concatenate((cards, flop, turn, river)),
                        np.concatenate((opponent_cards, flop, turn, river)),
                    )
                    if final_result == 1:
                        wins += 1
                        if initial_advantage == -1:
                            positive_potential += (
                                1  # won the hand with initial disadvantage
                            )
                        elif initial_advantage == 0:
                            positive_potential += 0.5
                    elif final_result == 0:  # draw
                        wins += 0.5
                        if initial_advantage == -1:
                            positive_potential += 0.5
                        elif initial_advantage == 1:
                            negative_potential += 0.5
                    else:
                        if (
                            initial_advantage == 1
                        ):  # lost the hand with initial advantage
                            negative_potential += 1
                        elif initial_advantage == 0:
                            negative_potential += 0.5
                    deck.return_cards(opponent_cards)
                    deck.return_cards(turn)
                    deck.return_cards(river)
                deck.return_cards(flop)
                effective_hand_strengths[
                    str(rank_one.value) + str(rank_two.value) + name + str(combination[0].value) + str(combination[1].value) + str(combination[2].value) + "m" + flush
                ] = (
                    wins * (iterations - negative_potential)
                    + (iterations - wins) * positive_potential
                ) / iterations**2
    return effective_hand_strengths

In [61]:
flop_effective_hand_strength_mono_and_flush = compute_mono_flop_effective_hand_strengths_for_suited_cards(10, made_flush=True)

In [62]:
for k,v in sorted(flop_effective_hand_strength_mono_and_flush.items(), key=lambda item: item[1], reverse=True):
    print(k, v)

AKsQJTmf 1.0
AKsQJ8mf 1.0
AKsQJ7mf 1.0
AKsQJ6mf 1.0
AKsQJ5mf 1.0
AKsQJ4mf 1.0
AKsQJ3mf 1.0
AKsQJ2mf 1.0
AKsQT9mf 1.0
AKsQT8mf 1.0
AKsQT7mf 1.0
AKsQT5mf 1.0
AKsQT4mf 1.0
AKsQT3mf 1.0
AKsQT2mf 1.0
AKsQ98mf 1.0
AKsQ97mf 1.0
AKsQ95mf 1.0
AKsQ94mf 1.0
AKsQ93mf 1.0
AKsQ92mf 1.0
AKsQ87mf 1.0
AKsQ84mf 1.0
AKsQ83mf 1.0
AKsQ82mf 1.0
AKsQ76mf 1.0
AKsQ74mf 1.0
AKsQ72mf 1.0
AKsQ65mf 1.0
AKsQ64mf 1.0
AKsQ63mf 1.0
AKsQ54mf 1.0
AKsQ53mf 1.0
AKsQ52mf 1.0
AKsQ43mf 1.0
AKsQ42mf 1.0
AKsQ32mf 1.0
AKsJT9mf 1.0
AKsJT7mf 1.0
AKsJT6mf 1.0
AKsJT3mf 1.0
AKsJT2mf 1.0
AKsJ97mf 1.0
AKsJ96mf 1.0
AKsJ95mf 1.0
AKsJ93mf 1.0
AKsJ92mf 1.0
AKsJ87mf 1.0
AKsJ86mf 1.0
AKsJ85mf 1.0
AKsJ84mf 1.0
AKsJ83mf 1.0
AKsJ82mf 1.0
AKsJ76mf 1.0
AKsJ75mf 1.0
AKsJ74mf 1.0
AKsJ73mf 1.0
AKsJ72mf 1.0
AKsJ65mf 1.0
AKsJ64mf 1.0
AKsJ63mf 1.0
AKsJ62mf 1.0
AKsJ54mf 1.0
AKsJ52mf 1.0
AKsJ43mf 1.0
AKsJ32mf 1.0
AKsT98mf 1.0
AKsT97mf 1.0
AKsT95mf 1.0
AKsT94mf 1.0
AKsT93mf 1.0
AKsT92mf 1.0
AKsT86mf 1.0
AKsT85mf 1.0
AKsT84mf 1.0
AKsT83mf 1.0
AKsT82mf 1.0

In [63]:
flop_effective_hand_strength_mono_and_nothing = compute_mono_flop_effective_hand_strengths_for_suited_cards(10, False)

In [64]:
for k,v in sorted(flop_effective_hand_strength_mono_and_nothing.items(), key=lambda item: item[1], reverse=True):
    print(k, v)

AKsAQ9mn 1.0
AKsA98mn 1.0
AKsA94mn 1.0
AKsA76mn 1.0
AKsKQ8mn 1.0
AKsKJ3mn 1.0
AQsAK5mn 1.0
AQsQ95mn 1.0
AJsAQJmn 1.0
AJsAJ8mn 1.0
ATsAK3mn 1.0
ATsAQ7mn 1.0
ATsA85mn 1.0
ATsA65mn 1.0
A9sA95mn 1.0
A8sAKTmn 1.0
A8sAQJmn 1.0
A8sA82mn 1.0
A8s863mn 1.0
A7sA62mn 1.0
A5sAJ5mn 1.0
A4sAJ4mn 1.0
A4sAT4mn 1.0
A2sA92mn 1.0
A2sA72mn 1.0
KQsK32mn 1.0
KQsJT9mn 1.0
KJsAKJmn 1.0
KJsK96mn 1.0
KTsAKTmn 1.0
K9sAK4mn 1.0
K9sKQ9mn 1.0
K9sK93mn 1.0
K9sK92mn 1.0
K8sK83mn 1.0
K8sK52mn 1.0
K7sK74mn 1.0
K6sKQ2mn 1.0
K4sKQ3mn 1.0
K4sK42mn 1.0
K3sKJ3mn 1.0
K3sK83mn 1.0
QJsAQ6mn 1.0
QJsKT9mn 1.0
QJsQJ5mn 1.0
QJsJ74mn 1.0
QTsQJ4mn 1.0
Q9sQ97mn 1.0
Q8sQ82mn 1.0
Q7sQ73mn 1.0
Q6sA62mn 1.0
Q6sQ65mn 1.0
Q6sQ64mn 1.0
Q5sQ53mn 1.0
JTsKJ8mn 1.0
JTsJ65mn 1.0
J9sKJ9mn 1.0
J9sQJ6mn 1.0
J9sJ92mn 1.0
J8sAJ8mn 1.0
J8sKJ8mn 1.0
J8sJ52mn 1.0
J7sKJ7mn 1.0
J7sQJ7mn 1.0
J7sJ87mn 1.0
J5sAJ5mn 1.0
J4sJ75mn 1.0
J3sJ53mn 1.0
J2sJ62mn 1.0
J2sJ32mn 1.0
T9sKT9mn 1.0
T9s963mn 1.0
T8sT83mn 1.0
T3sJT3mn 1.0
98s873mn 1.0
97sJ75mn 1.0
96sQ96mn 1.0

In [88]:
def create_a_lis_of_lower_ranks(rank: Rank) -> list:
    return [x for x in list(Rank) if x < rank]

In [137]:
def compute_21_flop_effective_hand_strengths_for_suited_cards(iterations: int, flush_draw=True):
    shift = 1
    name = "s"
    if flush_draw:
        flush = "d"
        second_suit = "c"
        third_suit = "s"
    else:
        flush = "n"
        second_suit = "s"
        third_suit = "d"
        
    effective_hand_strengths = {}
    for i, rank_one in enumerate(list(Rank)[::-1]):
        for rank_two in list(Rank)[::-1][i + shift :]:
            deck = Deck()
            cards = np.concatenate(
                (
                    deck.draw_card_by_tag(str(rank_one.value) + "c"),
                    deck.draw_card_by_tag(str(rank_two.value) + "c"),
                )
            )
            # mono and flush
            rank_list = list(Rank)
            if flush_draw:
                rank_list.remove(rank_one)
                rank_list.remove(rank_two)
            possible_combinations = itertools.combinations(rank_list[::-1], 2)
            for combination in possible_combinations:
                third_suit_rank_list = [combination[0], combination[1]] + create_a_lis_of_lower_ranks(combination[1])[::-1]
                if flush_draw:
                    if rank_two > combination[1]:
                        third_suit_rank_list = [rank_one, rank_two] + third_suit_rank_list
                    elif rank_one > combination[1]:
                        third_suit_rank_list = [rank_one] + third_suit_rank_list
                for rank in third_suit_rank_list:
                    wins = 0
                    positive_potential = 0
                    negative_potential = 0
                    flop = np.concatenate(
                        (
                            deck.draw_card_by_tag(str(combination[0].value) + second_suit),
                            deck.draw_card_by_tag(str(combination[1].value) + second_suit),
                            deck.draw_card_by_tag(str(rank.value) + third_suit),
                        )
                    )
                    flop = sort_hand(flop)
                    for _ in range(iterations):
                        deck.shuffle()
                        opponent_cards = deck.draw_cards(2)
                        initial_advantage = compare_hands_after_flop(
                            np.concatenate((cards, flop)),
                            np.concatenate((opponent_cards, flop)),
                        )
                        turn = deck.draw_cards(1)
                        river = deck.draw_cards(1)
                        final_result = compare_hands_after_flop(
                            np.concatenate((cards, flop, turn, river)),
                            np.concatenate((opponent_cards, flop, turn, river)),
                        )
                        if final_result == 1:
                            wins += 1
                            if initial_advantage == -1:
                                positive_potential += (
                                    1  # won the hand with initial disadvantage
                                )
                            elif initial_advantage == 0:
                                positive_potential += 0.5
                        elif final_result == 0:  # draw
                            wins += 0.5
                            if initial_advantage == -1:
                                positive_potential += 0.5
                            elif initial_advantage == 1:
                                negative_potential += 0.5
                        else:
                            if (
                                initial_advantage == 1
                            ):  # lost the hand with initial advantage
                                negative_potential += 1
                            elif initial_advantage == 0:
                                negative_potential += 0.5
                        deck.return_cards(opponent_cards)
                        deck.return_cards(turn)
                        deck.return_cards(river)
                    deck.return_cards(flop)
                    #if str(rank_one.value) + str(rank_two.value) + name + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + "t" + flush in effective_hand_strengths:
                    #    raise ValueError(str(rank_one.value) + str(rank_two.value) + name + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + "t" + flush)
                    effective_hand_strengths[
                        str(rank_one.value) + str(rank_two.value) + name + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + "t" + flush
                    ] = (
                        wins * (iterations - negative_potential)
                        + (iterations - wins) * positive_potential
                    ) / iterations**2
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strength_two_one_and_flush_draw = compute_21_flop_effective_hand_strengths_for_suited_cards(1000)

In [136]:
#for k,v in sorted(flop_effective_hand_strength_two_one_and_flush_draw.items(), key=lambda item: item[1], reverse=True):
#    print(k, v)
for k,v in flop_effective_hand_strength_two_one_and_flush_draw.items():
    print(k, v)

AKsAQJtd 0.81
AKsKQJtd 0.95
AKsQQJtd 0.6125
AKsQJJtd 0.955
AKsQJTtd 0.815
AKsQJ9td 0.925
AKsQJ8td 0.78
AKsQJ7td 0.88
AKsQJ6td 0.83
AKsQJ5td 0.7625
AKsQJ4td 0.8575
AKsQJ3td 0.96
AKsQJ2td 0.8575
AKsAQTtd 1.0
AKsKQTtd 0.905
AKsQQTtd 0.86
AKsQTTtd 0.7375
AKsQT9td 0.955
AKsQT8td 0.91
AKsQT7td 0.865
AKsQT6td 0.7525
AKsQT5td 0.9125
AKsQT4td 0.905
AKsQT3td 0.7725
AKsQT2td 0.865
AKsAQ9td 1.0
AKsKQ9td 0.85
AKsQQ9td 0.905
AKsQ99td 0.73
AKsQ98td 0.83
AKsQ97td 0.87
AKsQ96td 0.915
AKsQ95td 0.8
AKsQ94td 0.75
AKsQ93td 0.86
AKsQ92td 0.68
AKsAQ8td 0.9025
AKsKQ8td 0.855
AKsQQ8td 0.955
AKsQ88td 0.8575
AKsQ87td 0.915
AKsQ86td 0.81
AKsQ85td 0.865
AKsQ84td 0.8375
AKsQ83td 0.8575
AKsQ82td 1.0
AKsAQ7td 0.9025
AKsKQ7td 1.0
AKsQQ7td 0.91
AKsQ77td 0.765
AKsQ76td 0.73
AKsQ75td 0.68
AKsQ74td 0.905
AKsQ73td 0.675
AKsQ72td 0.81
AKsAQ6td 0.9025
AKsKQ6td 0.855
AKsQQ6td 0.85
AKsQ66td 0.905
AKsQ65td 0.7
AKsQ64td 0.775
AKsQ63td 0.915
AKsQ62td 0.955
AKsAQ5td 1.0
AKsKQ5td 0.9025
AKsQQ5td 0.7
AKsQ55td 0.73
AKsQ54td 0.76
AKsQ

In [ ]:
import json

In [ ]:
a = list(Rank)
a.remove(Rank.Two)
a.remove(Rank.Three)
for c in itertools.combinations(a[::-1], 2):
    print(c[0], c[1])
    

Rank.Ace Rank.King
Rank.Ace Rank.Queen
Rank.Ace Rank.Jack
Rank.Ace Rank.Ten
Rank.Ace Rank.Nine
Rank.Ace Rank.Eight
Rank.Ace Rank.Seven
Rank.Ace Rank.Six
Rank.Ace Rank.Five
Rank.Ace Rank.Four
Rank.King Rank.Queen
Rank.King Rank.Jack
Rank.King Rank.Ten
Rank.King Rank.Nine
Rank.King Rank.Eight
Rank.King Rank.Seven
Rank.King Rank.Six
Rank.King Rank.Five
Rank.King Rank.Four
Rank.Queen Rank.Jack
Rank.Queen Rank.Ten
Rank.Queen Rank.Nine
Rank.Queen Rank.Eight
Rank.Queen Rank.Seven
Rank.Queen Rank.Six
Rank.Queen Rank.Five
Rank.Queen Rank.Four
Rank.Jack Rank.Ten
Rank.Jack Rank.Nine
Rank.Jack Rank.Eight
Rank.Jack Rank.Seven
Rank.Jack Rank.Six
Rank.Jack Rank.Five
Rank.Jack Rank.Four
Rank.Ten Rank.Nine
Rank.Ten Rank.Eight
Rank.Ten Rank.Seven
Rank.Ten Rank.Six
Rank.Ten Rank.Five
Rank.Ten Rank.Four
Rank.Nine Rank.Eight
Rank.Nine Rank.Seven
Rank.Nine Rank.Six
Rank.Nine Rank.Five
Rank.Nine Rank.Four
Rank.Eight Rank.Seven
Rank.Eight Rank.Six
Rank.Eight Rank.Five
Rank.Eight Rank.Four
Rank.Seven Rank.Six


In [ ]:


with open("ehs.json", 'w') as f:
    json.dump(effective_hand_strengths, f)

In [ ]:
with open("waverage_strength.json", 'w') as f:
    json.dump(cards_strength, f)

In [19]:
with open("ehs.json", 'r') as f:
    cards_strength = json.load(f)

In [20]:
for k,v in sorted(cards_strength.items(), key=lambda item: item[1], reverse=True):
    print(k, v)

AAo 0.72504093115
KKo 0.68305080385
QQo 0.6488844208
54s 0.62793650675
65s 0.6257558772
76s 0.61852472975
53s 0.615117073125
64s 0.609163682
87s 0.6090829165
43s 0.608722533225
JJo 0.606115846975
75s 0.605527336425
98s 0.5988284916
86s 0.5968488013
52s 0.5948646311
42s 0.593194550125
63s 0.592384529175
32s 0.590392840475
74s 0.59015805285
T9s 0.5888193465
97s 0.5873583335
65o 0.586740245
54o 0.584789117
85s 0.5838598725
76o 0.5832674056
TTo 0.5775377743
87o 0.5767724845
T8s 0.57271780425
53o 0.57094552
JTs 0.5709050045
73s 0.570624126525
96s 0.570377951725
62s 0.57033191925
64o 0.569510728175
75o 0.56879458625
98o 0.56725181775
43o 0.567115602725
84s 0.56630449025
86o 0.562895108
T9o 0.56147025195
T7s 0.56061226115
72s 0.55846408525
95s 0.5582394128
J9s 0.5581327684
97o 0.5535281234
83s 0.5519820212
63o 0.55055055395
52o 0.5498880227
82s 0.54964569745
T6s 0.5483649832
74o 0.5471063256
JTo 0.54600591625
85o 0.545953219225
QJs 0.544317586
42o 0.544100115025
T8o 0.5430129977
32o 0.5429029

In [37]:
max_std = max([x[1] for x in cards_strength.values()])
min_std = min([x[1] for x in cards_strength.values()])

In [38]:
print(max_std, min_std)

1912.0 984.0


In [34]:
for k,v in cards_strength.items():
    cards_strength[k] = (v[0], round((v[1] - min_std) / (max_std - min_std), 3))

In [35]:
for k,v in sorted(cards_strength.items(), key=lambda item: item[1][0]):
    print(k,v)

AAo (2457.0, 0.0)
KKo (2599.0, 0.062)
QQo (2713.0, 0.142)
JJo (2825.0, 0.214)
TTo (2930.0, 0.297)
99o (3033.0, 0.377)
88o (3130.0, 0.458)
77o (3234.0, 0.545)
66o (3319.0, 0.627)
55o (3411.0, 0.718)
44o (3516.0, 0.811)
33o (3622.0, 0.903)
AKs (3671.0, 0.852)
AQs (3713.0, 0.856)
22o (3723.0, 0.998)
AJs (3737.0, 0.855)
ATs (3756.0, 0.859)
KQs (3764.0, 0.88)
JTs (3789.0, 0.905)
KJs (3795.0, 0.881)
QJs (3795.0, 0.897)
KTs (3800.0, 0.891)
QTs (3807.0, 0.903)
A9s (3830.0, 0.853)
A8s (3839.0, 0.852)
T9s (3846.0, 0.905)
AKo (3862.0, 0.721)
A7s (3864.0, 0.865)
A5s (3868.0, 0.897)
J9s (3869.0, 0.895)
Q9s (3881.0, 0.886)
K9s (3886.0, 0.876)
98s (3896.0, 0.904)
A6s (3903.0, 0.873)
AQo (3906.0, 0.726)
A4s (3907.0, 0.913)
T8s (3911.0, 0.899)
AJo (3933.0, 0.737)
J8s (3933.0, 0.891)
A3s (3938.0, 0.92)
87s (3943.0, 0.915)
ATo (3948.0, 0.736)
K7s (3950.0, 0.878)
KQo (3953.0, 0.786)
K8s (3953.0, 0.874)
97s (3954.0, 0.906)
Q8s (3956.0, 0.887)
KJo (3960.0, 0.789)
QJo (3961.0, 0.824)
A2s (3966.0, 0.95)
JTo (

In [56]:
deck = Deck()
deck.shuffle()
cards = np.concatenate((deck.draw_card_by_tag("Qc"), deck.draw_card_by_tag("Qs"), deck.draw_card_by_tag("Qd"), deck.draw_card_by_tag("Qh")))

In [57]:
sorted(cards, reverse=True, key=lambda x: (x, x.get_tag()))

[Queen of Clubs, Queen of Spades, Queen of Diamonds, Queen of Hearts]